# Lektion 8

In [ ]:
from sympy import *
init_printing()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Matrixplots

In [ ]:
i = Symbol('i')
j = Symbol('j')

In [ ]:
def hilbert_element(i,j):
    return 1/(i+j+1)

hilbert_element(i,j)

In [ ]:
hilbert = Matrix(20, 20, hilbert_element)

In [ ]:
#plt.imshow(hilbert)  # TypeError

In [ ]:
plt.imshow(np.array(hilbert).astype(float), interpolation='none')
plt.colorbar();

In [ ]:
plt.imshow(np.log(np.array(hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

In [ ]:
hilbert = Matrix(60, 60, hilbert_element)

In [ ]:
plt.imshow(np.log(np.array(hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

In [ ]:
inv_hilbert = hilbert**(-1)

In [ ]:
plt.imshow(np.log(np.array(inv_hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

Was ist das Problem?

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.add_subplot(121)
plt.imshow(np.log(abs(np.array(inv_hilbert).astype(float))), 
           interpolation='none')
plt.colorbar()
fig.add_subplot(122)
plt.imshow(np.sign(np.array(inv_hilbert).astype(float)), 
           interpolation='none')
plt.colorbar();

Als Ersatz für 3D-Plot:

In [ ]:
x = Symbol('x')
y = Symbol('y')
f = cos(sqrt(x**2+y**2))
fn = lambdify((x,y), f, 'numpy')

In [ ]:
x = np.linspace(-2*np.pi, 2*np.pi, 150)
y = x
X, Y = np.meshgrid(x, y)
W = fn(X, Y)
plt.imshow(W, origin='lower', cmap=plt.cm.coolwarm)
plt.colorbar();

In [ ]:
extent = (x[0], x[-1], y[0], y[-1])
extent

In [ ]:
W[W<0] = -W[W<0]  # abs(W) wäre einfacher

In [ ]:
plt.imshow(W, origin='lower', cmap=plt.cm.hot, extent=extent)
plt.colorbar();

## Lineare Gleichungssysteme

In [ ]:
A = Matrix(3, 3, range(1,10))
A

In [ ]:
b = Matrix([6, 15, 24])
b

Wir wollen $Ax=b$ lösen.

In [ ]:
x = Matrix([Symbol('x_'+str(j)) for j in [1,2,3]])
x

In [ ]:
glg = Eq(A*x, b)
glg

In [ ]:
Lsg = solve(glg)
Lsg

Probe:

In [ ]:
glg.subs(Lsg[0])

konkrete Lösung

In [ ]:
x.subs(Lsg[0]).subs(x[2], 0)

andere konkrete Lösung

In [ ]:
x.subs(Lsg[0]).subs(x[2], 1)

In [ ]:
solve(A*x, x)  # Kern von A

In [ ]:
solve(A*x, Matrix([0,0,1]))  # unlösbar

## Veränderliche und unveränderliche Matrizen

In [ ]:
B = A

In [ ]:
B[0,0] = 999
B

In [ ]:
A

In [ ]:
B = A.copy()
B[0,0] = 777
B

In [ ]:
A

In [ ]:
C = A[0:3, 0:3]
C

In [ ]:
C[1,1] = 555
C

In [ ]:
A

In [ ]:
type(A)

In [ ]:
Ai = simplify(A)
type(Ai)

In [ ]:
# Ai[0,0] = 3 # TypeError

In [ ]:
B = Matrix(A)
B[0,0] = 333
A, B

In [ ]:
type(Matrix(Ai))

## Parameterabhängige Matrizen

In [ ]:
a = Symbol('a')
A = Matrix(3,3,range(1,10))
A[0,0] = 1 + a
A

In [ ]:
glg = Eq(A*x, b)
glg

In [ ]:
solve(glg, x)

In [ ]:
glg = Eq(A*x, Matrix([0,0,1]))
glg

In [ ]:
solve(glg, x)

In [ ]:
A.det()

## Gauß-Elimination zu Fuß

In [ ]:
L, U, R = A.LUdecomposition()
L, U, R

In [ ]:
L*U == A  # aber nur, weil R=[]

In [ ]:
A = Matrix(3,3,range(1,10))
A[0,0] = 1+a
b = Matrix([0,0,1])
A1 = A.row_join(b)
A1

In [ ]:
def func1(b, j):
    return b/A1[0,0]
A1.row_op(0, func1)
A1

In [ ]:
def func2(b, j):
    return b - A[1,0]*A1[0,j]
A1.row_op(1, func2)
def func3(b, j):
    return b - A[2,0]*A1[0,j]
A1.row_op(2, func3)
A1

Das ist aber sehr fehleranfällig

In [ ]:
def mult_zeile(A, zeilennr, faktor):
    B = Matrix(A)
    def func(b, j):
        return b*faktor
    B.row_op(zeilennr, func)
    return B

In [ ]:
B = A.copy()
mult_zeile(B, 2, -3)

In [ ]:
def add_zeile(A, quell_nr, ziel_nr, faktor):
    B = Matrix(A)  
    def func(b, j):
        return b + B[quell_nr,j]*faktor
    B.row_op(ziel_nr, func)
    return B

In [ ]:
A1

In [ ]:
A2 = mult_zeile(A1, 1, 1/A1[1,1])
A2

In [ ]:
A3 = add_zeile(A2, 1, 2, -A2[2,1])
A3 = simplify(A3)
A3

In [ ]:
A4 = mult_zeile(A3, 2, 1/A3[2,2])
A4

In [ ]:
A5 = add_zeile(A4, 2, 1, -A4[1,2])
A5

In [ ]:
A6 = add_zeile(A5, 1, 0, -A5[0,1])
A7 = add_zeile(A6, 2, 0, -A6[0,2])
A7

In [ ]:
y = simplify(A7[0:3, 3])
y

In [ ]:
simplify(A*y) == b

## Eigenwerte

In [ ]:
A = Matrix(3, 3, range(1,10))
A.eigenvals()

In [ ]:
A.eigenvects()

In [ ]:
A.eigenvects(simplify=True)

In [ ]:
T, J = A.diagonalize()
T = simplify(T)
T, J

In [ ]:
simplify(T * J * T**(-1))

In [ ]:
B = Matrix(3, 3, [1,1,0,0,1,0,0,0,1])
B

In [ ]:
B.eigenvects()

In [ ]:
#B.diagonalize()   # MatrixError

In [ ]:
C = simplify(T**(-1)*B*T)
C

In [ ]:
T1, J = C.jordan_form()
J

In [ ]:
T1[0,0]

In [ ]:
T1 = simplify(T1)

In [ ]:
T1[0,0]

In [ ]:
simplify(T1 * J * T1**(-1) - C)

In [ ]:
Cn = np.array(C).astype(float)

In [ ]:
eigw, eigv = np.linalg.eig(Cn)
eigw

In [ ]:
eigv

In [ ]:
M = Matrix(eigv)
M.det()

Hier könnte man eine Aufgabe von machen